In [2]:
#import
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime
import pytz
import plotly.io as pio
import webbrowser
#open for browser
pio.renderers.default = "browser"

# data loading
df = pd.read_csv("Play Store Data.csv")

# clean installs coloumn
df["Installs"] = (
    df["Installs"]
    .astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)

df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")
df = df.dropna(subset=["Installs"])

#add countries
countries = [
    "India", "United States", "United Kingdom", "Canada",
    "Germany", "France", "Brazil", "Australia", "Japan", "South Africa"
]

df["Country"] = np.random.choice(countries, size=len(df))

# time condition
ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.now(ist).time()

start_time = datetime.strptime("6:00", "%H:%M").time()
end_time = datetime.strptime("23:00", "%H:%M").time()

if not (start_time <= current_time <= end_time):
    print("Dashboard not available outside 2 PM – 6 PM IST")

else:

    # filtering for category
    df_filtered = df[
        ~df["Category"].str.startswith(("A", "C", "G", "S"), na=False)
    ]

    top_categories = (
        df_filtered.groupby("Category")["Installs"]
        .sum()
        .sort_values(ascending=False)
        .head(5)
        .index
    )

    df_top5 = df_filtered[df_filtered["Category"].isin(top_categories)]

    df_map = (
        df_top5.groupby(["Country", "Category"], as_index=False)
        .agg({"Installs": "sum"})
    )
#highlighting
    df_map["Highlight"] = np.where(
        df_map["Installs"] > 1_000_000,
        "Above 1M",
        "Below 1M"
    )
#country with code iso
    country_iso = {
        "India": "IND",
        "United States": "USA",
        "United Kingdom": "GBR",
        "Canada": "CAN",
        "Germany": "DEU",
        "France": "FRA",
        "Brazil": "BRA",
        "Australia": "AUS",
        "Japan": "JPN",
        "South Africa": "ZAF"
    }

    df_map["ISO_Code"] = df_map["Country"].map(country_iso)

    # map -choropleth
    fig = px.choropleth(
        df_map,
        locations="ISO_Code",
        locationmode="ISO-3",
        color="Installs",
        hover_name="Country",
        hover_data=["Category", "Installs", "Highlight"],
        animation_frame="Category",
        color_continuous_scale="Viridis"
    )

    fig.update_layout(
        template="plotly_dark",
        width=1200,
        height=650,
        title={
            "text": "Global App Installs by Top 5 Categories (2 PM – 6 PM IST)",
            "x": 0.5,
            "font": {"size": 22}
        },
        font=dict(family="Arial", size=14),
        margin=dict(l=20, r=120, t=80, b=20),
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type="natural earth"
        ),
        coloraxis=dict(
            colorbar=dict(
                title="Total Installs",
                thickness=18,
                len=0.75
            )
        )
    )

    # save as html and opwn as browser   
    fig.write_html("PlayStore_Dashboard.html")
    webbrowser.open("PlayStore_Dashboard.html")

